In [ ]:
!pip install hopsworks

In [ ]:
!pip install modal-client

In [1]:
!modal token set --token-id ....

Verifying token against https://api.modal.com
Token verified successfully
Token written to /root/.modal.toml


In [ ]:
! pip install python-dotenv

In [4]:
import modal
import os

from API_call_AQI import get_air_quality_df
from API_call_visual_crossing import get_weather_df

LOCAL = True

if not LOCAL:
    
    stub = modal.Stub()
    image = modal.Image.debian_slim().pip_install(["hopsworks==3.0.4"])


    @stub.function(image=image, schedule=modal.Period(days=1), secrets=[modal.Secret.from_name("HOPSWORKS_API_KEY"),
                                                                        modal.Secret.from_name("AIR_QUALITY_API_KEY"),
                                                                        modal.Secret.from_name("WEATHER_API_KEY")])
    def f():
        g()


def g():
    import hopsworks
    import os
    from datetime import datetime
    project = hopsworks.login()
    fs = project.get_feature_store()

    city = 'Paris'
    date_today = datetime.now().strftime("%Y-%m-%d")

    AIR_QUALITY_API_KEY = os.getenv('AIR_QUALITY_API_KEY')
    WEATHER_API_KEY = os.getenv('WEATHER_API_KEY')

    df_air_quality = get_air_quality_df(city, AIR_QUALITY_API_KEY)
    df_weather = get_weather_df(city, date_today, WEATHER_API_KEY)

    air_quality_fg = fs.get_feature_group(
        name='air_quality_fg',
        version=1
    )
    air_quality_fg.insert(df_air_quality)

    weather_fg = fs.get_feature_group(
        name='weather_fg',
        version=1
    )
    weather_fg.insert(df_weather)


if __name__ == "__main__":
    if LOCAL:
        from dotenv import load_dotenv

        load_dotenv()
        g()
    else:
        with stub.run():
            f()


Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/5315
Connected. Call `.close()` to terminate connection gracefully.


Uploading Dataframe: 0.00% |          | Rows 0/1 | Elapsed Time: 00:00 | Remaining Time: ?

Launching offline feature group backfill job...
Backfill Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/5315/jobs/named/air_quality_fg_1_offline_fg_backfill/executions


Uploading Dataframe: 0.00% |          | Rows 0/1 | Elapsed Time: 00:00 | Remaining Time: ?

Launching offline feature group backfill job...
Backfill Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/5315/jobs/named/weather_fg_1_offline_fg_backfill/executions
